In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import random

In [3]:
civil_df_data = pd.read_csv("./data/all_data_with_identities.csv")
train_identities_df = pd.read_csv("./data/train_identities.txt", header=None)
train_identities_lists = train_identities_df.values.tolist()
test_identities_df = pd.read_csv("./data/test_identities.txt", header=None)
test_identities_lists = test_identities_df.values.tolist()

In [4]:
test_identities = []
for ident in test_identities_lists:
    test_identities.append(ident[0])

In [5]:
df_non_toxic = civil_df_data[civil_df_data['toxicity']<0.5]

In [7]:
sentences = []
a = []
toxicity = []

for row_index in tqdm(range(len(civil_df_data))):
    comment_text = civil_df_data.iloc[row_index]['comment_text'].split()
    if civil_df_data.iloc[row_index]['toxicity'] < 0.5 and len(comment_text) <= 10 and len(set(test_identities).intersection(comment_text)) != 0:
        identity = str(set(test_identities).intersection(comment_text).pop())
        sentences.append(civil_df_data.iloc[row_index]['comment_text'])
        toxicity.append(0)
        cur_a = []
        
        for diff_identity in test_identities:
            cur_a.append(civil_df_data.at[row_index, "comment_text"].replace(identity, diff_identity))
        a.append(cur_a)

100%|██████████| 448000/448000 [02:11<00:00, 3411.90it/s]


In [8]:
civil_df_raw_test = pd.DataFrame(list(zip(*a)))
civil_df_test = civil_df_raw_test.T

civil_df_test.insert(0, column='comment_text', value=sentences)

In [9]:
civil_df_test.to_csv(Path("./data/civil_folder/civil_test_data.csv"))